# Retail Demo Store - Personalization Workshop - Lab 4

In this lab we are going to build on the [prior lab](./Lab-03-(Core)-Create-recommenders-and-custom-solutions.ipynb) by creating Amazon Personalize domain recommenders and custom solutions for additional use cases.

## Lab 4 Objectives

In this lab we will accomplish the following steps.

- Evaluate the recommendations from the e-commerce recommenders created in the last lab.
- Evaluate the recommendations from the custom solutions and campaigns created in the last lab.
- Activate the recommenders and campaigns in the Retail Demo Store storefront by setting their ARNs in the System Manager Parameter Store.
- Real-time events:
    - Create a Amazon Personalize Event Tracker that can be used to stream real-time events in the storefront to Personalize so Personalize can learn from user bahvior in real-time.
    - Evaluate the effect of the event tracker on real-time recommendations.
    - Configure and deploy the Retail Demo Store web app to pick up the event tracker so it can start streaming events.
- Create and evaluate how to use filters to apply business rules to recommendations and to promote a specific set of items while maintaining relevance.

This lab should take 30-45 minutes to complete.

## Setup

Just as in the previous labs, we have to prepare our environment by importing dependencies and creating clients.

### Import dependencies

The following libraries are needed for this lab.

In [ ]:
import boto3
import json
import time
import requests
import random
import uuid
import pandas as pd
from IPython.display import Image, HTML

from botocore.exceptions import ClientError

### Create clients

We will need the following AWS service clients in this lab. Notice that we are creating some new Personalize clients with the service name of `personalize-runtime` and `personalize-events`. We'll be using these clients in this lab to get recommendations from our recommenders and campaigns and sending events to Personalize.

In [ ]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
personalize_events = boto3.client('personalize-events')
servicediscovery = boto3.client('servicediscovery')
ssm = boto3.client('ssm')

### Load variables saved in prior labs

At the end of Lab 1 we saved some variables that we'll need in this lab. The following cell will load those variables into this lab environment.

In [ ]:
%store -r

### Lookup IP addresses of Products and Users microservices

In this lab we will need to lookup details on recommended products and users. We'll do this by making RESTful API calls to these services. In the cells below, we will lookup the IP addresses of these microservices using [AWS Cloud Map](https://aws.amazon.com/cloud-map/)'s Service Discovery.

In [ ]:
response = servicediscovery.discover_instances(
    NamespaceName='retaildemostore.local',
    ServiceName='products',
    MaxResults=1,
    HealthStatus='HEALTHY'
)

assert len(response['Instances']) > 0, 'Products service instance not found; check ECS to ensure it launched cleanly'

products_service_instance = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
print('Products Service Instance IP: {}'.format(products_service_instance))

In [ ]:
response = requests.get('http://{}/products/all'.format(products_service_instance))
products = response.json()
products_df = pd.DataFrame(products)
products_df.head(5)

In [ ]:
response = servicediscovery.discover_instances(
    NamespaceName='retaildemostore.local',
    ServiceName='users',
    MaxResults=1,
    HealthStatus='HEALTHY'
)

assert len(response['Instances']) > 0, 'Users service instance not found; check ECS to ensure it launched cleanly'

users_service_instance = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
print('Users Service Instance IP: {}'.format(users_service_instance))

In [ ]:
response = requests.get('http://{}/users/all?count=10000'.format(users_service_instance))
users = response.json()
users_df = pd.DataFrame(users)
users_df.head(5)

### Load interactions dataset

Next let's load the interaction dataset (the CSV created in Lab 1) so we can query it to see what historical interactions were used to train the model for each user. This will help us better understand why certain products are being recommended.

In [ ]:
interactions_df = pd.read_csv(interactions_filename)
interactions_df['USER_ID'] = interactions_df.USER_ID.astype(str)
interactions_df['TIMESTAMP'] = pd.to_datetime(interactions_df['TIMESTAMP'],unit='s')
interactions_df.head(10)

Next let's create a couple utility functions that we can use later in the notebook to lookup recent interactions and product details for past interactions.

The first function will lookup the most recent interactions for a user and return them in a dataframe.

In [ ]:
# Update DF rendering
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_colwidth', None)

def lookup_historical_interactions(user_id, max_count = 10):
    recent_df = interactions_df.loc[interactions_df['USER_ID'] == str(user_id)]
    recent_df = recent_df.sort_values(by = 'TIMESTAMP', ascending = False)
    recent_df = recent_df[:max_count]
    
    rows = []
    columns_to_keep = ['id', 'name', 'category', 'style', 'price', 'image']
    for index, interaction in recent_df.iterrows():
        product = products_df.loc[products_df['id'] == interaction['ITEM_ID']]
        if product.empty:
            continue
        product = product.iloc[0]
        row = {}
        row['TIMESTAMP'] = interaction['TIMESTAMP']
        row['EVENT_TYPE'] = interaction['EVENT_TYPE']
        for col in columns_to_keep:
            if col == 'image':
                row[col] = '<img src="' + product[col] + '"/>'
            elif col == 'name':
                row[col] = '<b>' + product[col] + '</b>'
            else:
                row[col] = product[col]
        rows.append(row)
    
    return pd.DataFrame(rows)


Finally, let's test the interaction history lookup function for a random user.

In [ ]:
# Randomly select a user.
user = users_df.sample(1).iloc[0]
user_id = user['id']
# Lookup recent interactions and product details for user.
df = lookup_historical_interactions(user_id, 20)
# Display info on user and recent interactions
header = f'<h3>Recent interactions for {user["first_name"]} {user["last_name"]} (#{user_id})</h3>'
header += f'<h4>Persona: {", ".join(user["persona"].split("_"))}</h4>'
HTML(header + df.to_html(escape=False))

## Evaluate Recommenders

Now let's evaluate the product recommendations from the recommenders we created in the last lab for our random user.

### Recommended For You recommender

We'll start with the "Recommended For You" recommender that we created in the last lab. This recommender provides personalized product recommendations for a specific user. We'll use the same random user selected above.

Taking note of the recent interactions and shopper persona above, let's retrieve recommendations for this user from the Recommended For You recommender.

> As a reminder, the shopper persona was used to generate interaction history for the user by creating interactions (clicks, purchases, etc) against products in the categories represented in the persona. Since the model is trained based on these interactions we should expect to see recommendations that are consistent with the persona.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = rfy_recommender_arn,
    userId = str(user_id),
    numResults = 10
)

item_list = get_recommendations_response['itemList']
print(json.dumps(item_list, indent=4))

As you can see, Personalize only returns an `itemId` for each recommended item. Since the `itemId` alone doesn't tell us much about each product, let's enhance the notebook display output to include more details about each product.

The following code cell declares a helper function that will call `GetRecommendations` on a recommender or campaign, lookup details on each recommended item by calling the Products microservice, and return a dataframe that we can use to display the results. Be sure to execute the cell below so the function is created.

In [ ]:
def get_recommendations_as_df(inference_arn, user_id = None, item_id = None, num_results = 15, filter_arn = None, 
                              filter_values = None, promotion = None):
    params = {
        'numResults': num_results
    }
    if user_id:
        params['userId'] = user_id
    if item_id:
        params['itemId'] = item_id
    if filter_arn:
        params['filterArn'] = filter_arn
        if filter_values:
            params['filterValues'] = filter_values
    if promotion:
        params['promotions'] = [ promotion ]
        
    is_recommender = inference_arn.split(':')[5].startswith('recommender/')
    if is_recommender:
        params['recommenderArn'] = inference_arn
    else:
        params['campaignArn'] = inference_arn

    get_recommendations_response = personalize_runtime.get_recommendations(**params)
        
    item_list = get_recommendations_response['itemList']
    columns_to_keep = ['id', 'name', 'category', 'style', 'price', 'image', 'description', 'promoted', 'gender_affinity']
    recommendation_list = []
    for item in item_list:
        product = products_df.loc[products_df['id'] == item['itemId']]
        if product.empty:
            continue
        product = product.iloc[0]
        row = {}
        for col in columns_to_keep:
            if col == 'image':
                row[col] = '<img src="' + product[col] + '"/>'
            elif col == 'name':
                row[col] = '<b>' + product[col] + '</b>'
            else:
                row[col] = product[col]
        recommendation_list.append(row)

    return pd.DataFrame(recommendation_list)

Now let's test the function by fetching recommendations for the same user again and displaying the dataframe as HTML.

In [ ]:
df = get_recommendations_as_df(rfy_recommender_arn, user_id = user["id"], num_results = 15)
header = f'<h3>Recommended-For-You recommendations for {user["first_name"]} {user["last_name"]} (#{user["id"]})</h3>'
header += f'<h4>Persona: {", ".join(user["persona"].split("_"))}</h4>'
HTML(header + df.to_html(escape=False))

Are the recommended products consistent with the shopper's interaction history and persona?

### Most Viewed recommender

Next let's take a look at the recommendations from the "Most Viewed" recommender. This recommender looks at product view interactions across all users to find the most popular products. Therefore, items recommended by this recommender will be the same across all users. In other words, they're not personalized to the user but rather a representation of what's popular based on all user behavior. This recommender will be used in the storefront to make recommendations of popular items to brand new/cold users.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = most_viewed_recommender_arn,
    userId = str(user_id),
    numResults = 10
)

item_list = get_recommendations_response['itemList']
print(json.dumps(item_list, indent=4))

As before, let's lookup details on each of the products to provide a better sense of what's popular for the storefront.

In [ ]:
df = get_recommendations_as_df(most_viewed_recommender_arn, user_id = user["id"], num_results = 15)
header = f'<h3>Most Viewed recommendations for {user["first_name"]} {user["last_name"]} (#{user["id"]})</h3>'
header += f'<h4>Persona: {", ".join(user["persona"].split("_"))}</h4>'
HTML(header + df.to_html(escape=False))

Notice how the recommendations are completely different. This is because the Most Viewed recipe makes recommendations based on popularity of items across all users. Therefore, the Most Viewed recommendations are not personalized to the individual user and will be the same for all users. This makes it useful for use cases such as getting cold users engaged with your products. For this retail demo scenario, the convenience store products (pizza, soda, chips, etc) are actuall the most popular items across the diverse catalog. We'll see later in this lab how filters can be used to constrain recommendations to a portion of the catalog or based on the current user's interaction history. 

## Evaluate custom campaigns

As you may recall from the prior lab, we created custom solutions for the related items and personalized ranking use cases. Let's evaluate the recommendations from the campaigns for those solutions below.

### Similar items custom campaign

The [Similar-Items](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-similar-items.html) recipe is designed to balance co-interactions across all users and thematic similarity between items to make relevant related items recommendations. Since the input for related items recommendations is an item ID, let's select a product from the catalog to use as our source item.

In [ ]:
product = products_df.sample(1)
product

Now let's get some related item recommendations from the Similar Items based campaign for the above product. Notice that we're using the same `GetRecommendation` API as the recommenders above but this time we're specifying a `campaignArn` rather than a `recommenderArn`.

In [ ]:
product_id = product.iloc[0]['id']

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = similar_items_campaign_arn,
    itemId = str(product_id),
    numResults = 10
)

item_list = get_recommendations_response['itemList']
print(json.dumps(item_list, indent=4))

As before, we'll lookup the product details for each recommended product.

In [ ]:
df = get_recommendations_as_df(similar_items_campaign_arn, item_id = product_id, num_results = 15)
header = f'<h3>Similar Items for {product.iloc[0]["name"]} in {product.iloc[0]["category"]} (#{product.iloc[0]["id"]})</h3>'
HTML(header + df.to_html(escape=False))

How are the related item recommendations? Is there room for improvement in keeping them thematically similar? Since the Similar-Items recipe is looking at co-interactions as well as item metadata, this recipe naturally provides recommendations of items that you may also like rather than purely similar items. Later in this lab we'll see how filters can be used to setup thematic guardrails for similar items.

### Personalized Ranking campaign

Next let's evaluate the results of the personalized ranking campaign. As a reminder, given a list of items and a user, this campaign will rerank the items based on the preferences of the user. For the Retail Demo Store, we will use this campaign to rerank the products listed for each category and the featured products list as well as reranking catalog search results displayed in the search widget.

#### Get Featured Products List

First let's get the list of featured products from the Products microservice and display the raw JSON response.

In [ ]:
response = requests.get('http://{}/products/featured'.format(products_service_instance))
featured_products = response.json()
print(json.dumps(featured_products, indent = 4))

#### ReRank Featured Products

Using the featured products list just retrieved above, we'll isolate the item IDs so we can use them to test reranking for a user. We'll also prepare a dataframe that we we can use to compare the the rerank list. This reranking will allow us to provide ranked products based on the user's behavior. These behaviors should be consistent the same persona that was mentioned above (since we're going to use the same `user_id`).

In [ ]:
unranked_product_ids = []
unranked_products = []

for product in featured_products:
    unranked_product_ids.append(product['id'])
    unranked_products.append(f'<b>{product["name"]}</b><br/>{product["category"]}/{product["style"]}<br/>{product["id"]}<br/><img src="{product["image"]}" width="150" align="right"/>')

unranked_products_df = pd.DataFrame(unranked_products, columns = ["Unranked products"])
HTML(unranked_products_df.to_html(escape=False))

Now let's have Personalize rank the featured product IDs based on our random user. We'll first display the raw response from the GetPersonalizedRanking API.

In [ ]:
response = personalize_runtime.get_personalized_ranking(
    campaignArn=ranking_campaign_arn,
    inputList=unranked_product_ids,
    userId=str(user["id"])
)
reranked = response['personalizedRanking']
print(json.dumps(response['personalizedRanking'], indent = 4))

To make the unranked and rerank list of products easier to compare, let's create another dataframe of the reranked items and dispay the two dataframes side by side.

In [ ]:
reranked_products = []

for item in reranked:
    product = products_df.loc[products_df['id'] == item['itemId']]
    if product.empty:
        continue
    product = product.iloc[0]
    reranked_products.append(f'<b>{product["name"]}</b><br/>{product["category"]}/{product["style"]}<br/>{product["id"]}<br/><img src="{product["image"]}" width="150" align="right"/>')

reranked_products_df = pd.DataFrame(reranked_products, columns = [ 'Reranked products' ])

df = pd.concat([unranked_products_df, reranked_products_df], axis=1)

header = f'<h3>Unranked and Reranked products for {user["first_name"]} {user["last_name"]} (#{user["id"]})</h3>'
header += f'<h4>Persona: {", ".join(user["persona"].split("_"))}</h4>'
HTML(header + df.to_html(escape=False))

Are the reranked results for our user different than the original results from the Products service? Does the reranked list more closely reflect the interests in the user's persona? Experiment with a different user in the cells above to see how the item ranking changes.

### Pick products for discount - contextual recommendations

Using the featured products list we'll pick some products for discount from the featured products.

We'll get the ranking when discount context is applied for comparison. This is a using the "contextual metadata" feature of Amazon Personalize.

In [ ]:
response = personalize_runtime.get_personalized_ranking(
    campaignArn=ranking_campaign_arn,
    inputList=unranked_product_ids,
    userId=str(user_id),
    context={'DISCOUNT': 'Yes'} # Here we provide the context for the ranking
)
discount_reranked = response['personalizedRanking']
print('Discount context ranking:', json.dumps(discount_reranked, indent = 4))
print('Discount:', [item['itemId'] for item in discount_reranked[:2]])

We could use the discount-context ranking directly, but what we might be more interested in seeing is those products that benefit from having a discount shown. In our simulated data, certain products are more likely to see purchases with discount (to be precise, the less expensive ones). Let us find out which products benefit most. We also make use of the scores returned by Personalize when it returns the ranking.

In [ ]:
eps = 0.00001 #  "epsilon" - a number slightly more than zero so we don't get division by zero
non_discount_rerank_scores = {item['itemId']: max(item['score'], eps) for item in reranked}
discount_rerank_scores = {item['itemId']: item['score'] for item in discount_reranked}
score_increases_with_discount = {item_id: discount_rerank_scores[item_id]/non_discount_rerank_scores[item_id]
                                 for item_id in discount_rerank_scores}
# Let us get the sorted items:
discount_improve_sorted_items = sorted(score_increases_with_discount.keys(),
                                       key=lambda key: score_increases_with_discount[key])

print('Improvement ranking:', discount_improve_sorted_items)
# Let us pick the two items that respond best to discounts
print('Discount:', discount_improve_sorted_items[:2])

Has the ranking changed?

## Enable recommenders and campaigns in Retail Demo Store Recommendations service

Now that we've tested our campaigns and can get related product, product recommendations, and reranked items for our users, we need to enable the recommenders and campaigns in the Retail Demo Store's [Recommendations service](https://github.com/aws-samples/retail-demo-store/tree/master/src/recommendations). The Recommendations service is called by the Retail Demo Store Web UI when a user visits a page with personalized content capabilities (home page, product detail page, and category page). The Recommendations service checks Systems Manager Parameter values to determine the Personalize recommender and campaign ARNs to use for each of our personalization use-cases.

Let's set the recommender and campaign ARNs in the expected parameter names.

### Update SSM Parameters to enable recommenders

In [ ]:
response = ssm.put_parameter(
    Name='/retaildemostore/personalize/recommended-for-you-arn',
    Description='Retail Demo Store Recommended For You Recommender/Campaign Arn Parameter',
    Value='{}'.format(rfy_recommender_arn),
    Type='String',
    Overwrite=True
)

In [ ]:
response = ssm.put_parameter(
    Name='/retaildemostore/personalize/popular-items-arn',
    Description='Retail Demo Store Most Viewed Recommender/Campaign Arn Parameter',
    Value='{}'.format(most_viewed_recommender_arn),
    Type='String',
    Overwrite=True
)

### Update SSM Parameter to enable campaigns

In [ ]:
response = ssm.put_parameter(
    Name='/retaildemostore/personalize/related-items-arn',
    Description='Retail Demo Store Also Viewed Recommender/Campaign Arn Parameter',
    Value='{}'.format(similar_items_campaign_arn),
    Type='String',
    Overwrite=True
)

In [ ]:
response = ssm.put_parameter(
    Name='/retaildemostore/personalize/personalized-ranking-arn',
    Description='Retail Demo Store Personalized Ranking Campaign Arn Parameter',
    Value='{}'.format(ranking_campaign_arn),
    Type='String',
    Overwrite=True
)

## Evaluate Personalization in Retail Demo Store's Web UI

Now that you've enabled each personalization feature by setting the respective recommender and campaign ARN, you can test these personalization features through the Retail Demo Store's Web App UI. If you haven't already opened a browser window/tab to the Retail Demo Store Web UI, navigate to the CloudFormation console in this AWS account and check the Outputs section of the stack used to launch the Retail Demo Store. Make sure you're checking the base/root stack and not the nested stacks that were created. In the Outputs section look for the output named: WebURL and browse to the URL provided.

![CloudFormation Outputs](../images/cfn-webui-outputs.png)

If you haven't already created a user account in your Retail Demo Store instance, let's create one now. When you access the Retail Demo Store Web UI for the first time, you will be prompted to create an account or sign in to an existing account. Click the "**Create an account**" button. If you skipped the account creation process, click the "**Sign In**" button and then click the "**No account? Create account**" link to create an account. Follow the prompts and enter the required data. You will need to provide a valid email address in order to receive an email with the confirmation code to validate your account.

Once you've created and validated your account, click on the Sign In button again and sign in with the account you created.

### Emulate Shopper

To confirm product recommendations are personalized, you can emulate one of the many ficticious shoppers loaded into the system. You can also switch between shoppers by clicking the shopper profile name and details in the top navigation. You can have a shopper auto-selected for you or you can choose your own. In the shopper selection modal dialog, specify an age range and a primary shopping interest. Click Submit and a closely matching shopper is shown, confirm your choice or try again. Product recommendations should match the persona of the shopper you've selected.

### Viewing Related Product Recommendations

Let's start with the Related Product Recommendations use-case.
This recommender for this use-case is based on the [Similar-Items](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-similar-items.html)
custom solution recipe which uses item-to-item co-interactions and thematic item similarity (based on item metadata) to determine item similarity.

Browse to a [product detail page](https://github.com/aws-samples/retail-demo-store/blob/master/src/web-ui/src/public/ProductDetail.vue)
and evaluate the products listed in the **Compare similar items** section.
You should see the Personalize service icon displayed to the right of the section header.
This tells you that results are actually coming from a recommender or campaign.
If you don't see the Personalize service icon and recipe name, the page is using default behavior of displaying products from the same category
(verify that the campaign was created successfully above **and** the campaign ARN is set as an SSM parameter).

![Related Product Recommendations](images/retaildemostore-related-products.jpg)

### Viewing Product Recommendations

With the user emulation saved, browse to the Retail Demo Store
[home page](https://github.com/aws-samples/retail-demo-store/blob/master/src/web-ui/src/public/Main.vue) and evaluate
the products listed in the **Inspired by your shopping trends** section.
Do they appear consistent with the shopping persona you're emulating? For the screenshots listed here,
the user was trained with historical data based primarily on products from the "Footwear" category, then to a lesser degree on products from the "Jewelry" category, and slightly on products from the "Furniture" category.

![Personalized Product Recommendations](images/retaildemostore-product-recs.jpg)

Note that if the section is titled **Featured** or you don't see the Personalize service icon and recipe name displayed, this indicates that either you are not signed in as a user or the recommender ARN is not set as the appropriate SSM parameter. Double check that the recommender was created successfully in the prior lab and that the recommender ARN is set in SSM.

### Personalized Ranking

Finally, let's evaluate the personalizated ranking use-case.
There are two places where personalized ranking is implemented in the Retail Demo Store.
With a user emulated, browse to the featured product category list by clicking on "Featured" from the Retail Demo Store home page.
Note how for the emulated user with a persona of "Electronics, Outdoors, Footwear" has the headphones, frisbee, and pair of shoes sorted to the top of the list.
(See [CategoryDetail.vue](https://github.com/aws-samples/retail-demo-store/blob/master/src/web-ui/src/public/CategoryDetail.vue)).

![Personalized Product Ranking](images/retaildemostore-personalized-ranking.jpg)

The other feature where personalized ranking is implemented is in
[search results](https://github.com/aws-samples/retail-demo-store/blob/master/src/web-ui/src/public/Search.vue).
Start typing a word in the search box and a search result widget will be displayed.
If the results were reranked by Personalize, you will see a "Personalize Ranking" annotation in the search box.
For the emulated user with a historical affinity for electronics, outdoors, and footwear,
notice that a search for product keywords starting with "s" will move _shoes_ and _speakers_ to the top of the results.

![Personalized Search Results](images/retaildemostore-personalized-search.jpg)

If the search functionality is not working at all for you, make sure that you
completed the [Search workshop](../0-StartHere/Search.ipynb).

### Personalized Discounts

The Personalized Discounts are enabled against the [Amazon Interactive Video Service](https://aws.amazon.com/ivs/) (IVS) demo. You can access this page from the "Shop" dropdown and then select "Live Streams" in the navigation bar. Discounted products are chosen over the current set of products streamed from the IVS live stream:

![Personalized Discounts](images/retaildemostore-personalized-discounts.png)

Currently 2 products are selected for each video to be offered a discount.


## Event Tracking - Keeping up with evolving user intent

Up to this point we have trained and deployed Amazon Personalize recommenders and campaigns based on historical data that we generated in this workshop. This allows us to make related product, user recommendations, and rerank product lists based on already observed behavior of our users. However, user intent often changes in real-time such that what products the user is interested in now may be different than what they were interested in a week ago, a day ago, or even a few minutes ago. Making recommendations that keep up with evolving user intent is one of the more difficult challenges with personalization. Fortunately, Amazon Personalize has a mechanism for this exact issue.

Amazon Personalize supports the ability to send real-time user events (i.e. clickstream) data into the service.
Personalize uses this event data to adjust recommendations. It will also save these events and automatically
include them when recommenders and solutions for the same dataset group are re-trained.

The Retail Demo Store's Web UI already has
[logic to send events](https://github.com/aws-samples/retail-demo-store/blob/master/src/web-ui/src/analytics/AnalyticsHandler.js)
such as 'View', 'AddToCart', 'Purchase', and others as they occur in real-time to a Personalize Event Tracker.
These are the same event types we used to initially create the recommenders, solutions, and campaigns for our personalization use-cases.
All we need to do is create an event tracker in Personalize, set the tracking Id for the tracker in an SSM parameter,
and rebuild the Web UI service to pick up the change.

### Create Personalize Event Tracker

Let's start by creating an event tracker for our dataset group.

In [ ]:
try:
    event_tracker_response = personalize.create_event_tracker(
        datasetGroupArn=dataset_group_arn,
        name='retaildemostore-event-tracker'
    )

    event_tracker_arn = event_tracker_response['eventTrackerArn']
    event_tracking_id = event_tracker_response['trackingId']
except personalize.exceptions.ResourceAlreadyExistsException:
    print('You aready created an event tracker for this dataset group, seemingly')
    paginator = personalize.get_paginator('list_event_trackers')
    for paginate_result in paginator.paginate(datasetGroupArn = dataset_group_arn):
        for event_tracker in paginate_result['eventTrackers']:
            if event_tracker['name'] == 'retaildemostore-event-tracker':
                event_tracker_arn = event_tracker['eventTrackerArn']
                
                response = personalize.describe_event_tracker(eventTrackerArn = event_tracker_arn)
                event_tracking_id = response['eventTracker']['trackingId']
                break

print('Event Tracker ARN: ' + event_tracker_arn)
print('Event Tracking ID: ' + event_tracking_id)

### Wait for Event Tracker Status to Become ACTIVE

The event tracker should take a minute or so to become active.

In [ ]:
status = None
max_time = time.time() + 60*60 # 1 hours
while time.time() < max_time:
    describe_event_tracker_response = personalize.describe_event_tracker(
        eventTrackerArn = event_tracker_arn
    )
    status = describe_event_tracker_response["eventTracker"]["status"]
    print("EventTracker: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

### Update SSM Parameter To Enable Event Tracking

The Retail Demo Store's Web UI service just needs a Personalize Event Tracking Id to be able to send events to Personalize. The CodeBuild configuration for the Web UI service will pull the event tracking ID from an SSM parameter. 

Let's set our tracking ID in an SSM parameter.

In [ ]:
response = ssm.put_parameter(
    Name='/retaildemostore/personalize/event-tracker-id',
    Description='Retail Demo Store Personalize Event Tracker ID Parameter',
    Value='{}'.format(event_tracking_id),
    Type='String',
    Overwrite=True
)

### Trigger Web UI Service Release

Next let's trigger a new release of the Retail Demo Store's Web UI service so that it will pick up our SSM parameter change.

In the AWS console, browse to the AWS Code Pipeline service. Find the pipeline with **WebUIPipeline** in the name. Click on the pipeline name.

![AWS CodePipeline](images/retaildemostore-codepipeline.png)

#### Trigger Release

To manually trigger a new release, click the **Release change** button, click the **Release** button on the popup dialog window, and then wait for the pipeline to build and deploy. This will rebuild the web app, deploy it to the web UI S3 bucket, and invalidate the CloudFront distribution to force browsers to load from the origin rather than from their local cache.

![AWS CodePipeline Release](images/retaildemostore-codepipeline-release.png)

### Verify Event Tracking

Return to your web browser tab/window where the Retail Demo Store Web UI is loaded and **reload the web app/page**. **Reloading the page is important so that the web app is reloaded in your browser and the new event tracking configuration is loaded as well.**

There are a couple ways to verify that events are being sent to the Event Tracker. First, you can use your browser's Developer Tools to monitor the network calls made by the Retail Demo Store Web UI when you're browsing to product detail pages, adding items to carts, and completing orders. The other way you can verify that events are being received by the event tracker is in CloudWatch metrics for Personalize.

1. If you have done so, **reload the web app by refreshing/reloading your browser page.** This is important so you browser session picks up the Event Tracker change released above.
2. If not already signed in as a storefront user, sign in as (or create) a user. 
3. In the Retail Demo Store Web app, view product detail pages, add items to your cart, complete an order.
4. Verify that the Web UI is making "events" calls to the Personalize Event Tracker.
5. In the AWS console, browse to CloudWatch and then Metrics.

![Personalize CloudWatch Metrics](images/retaildemostore-eventtracker-cw.png)

If events are not being sent to the event tracker, make sure that the WebUIPipeline pipeline was built and deployed successfully and that you reloaded the web app in your browser. Note that it make take a minute or so before events are reflected in CloudWatch.

To assess the impact of real-time event tracking in recommendations made by the user recommendations on the home page, follow these steps.

1. Sign in as (or create) a storefront user.
2. View the product recommendations displayed on the home page under the "Inspired by your shopping trends" header. Take note of the products being recommended.
3. View products from categories that are not being recommended by clicking on their product images to take you to the product detail view. When you view the details for a product, an event is fired and sent to the Personalize event tracker.
4. Return to the home page and you should see product recommendations subtly changing to reflect the products you've engaged with. Repeat this process with other products and return to home page to see how recommendations are shifting.

### Cold User Recommendations

One of the key features of Personalize is being able to cold start users. Cold users are typically those who are new to your site or application and cold starting a user is getting from no personalization to making personalized recommendations in real-time. 

Personalize accomplishes cold starting users via the Event Tracker, just as we saw above with existing users. However, since new users are typically anonymous for a period of time before they create an account or may choose to transact as a guest, personalization is a valuable tool to help convert those anonymous users to transacting users. 

The challenge here is that Personalize needs a `userId` for anonymous users before it can make personalized recommendations. The Retail Demo Store solves this challenge by creating a provisional user ID the moment an anonymous user first hits the site. This provisional user ID is then used when streaming events to the Event Tracker and when retrieving recommendations from the Recommendations service. This allows the Retail Demo Store to start serving personalized recommendations after the first couple events are streamed to Personalize. Before recommendations can be personalized, Personalize will provide recommendations for popular items as a fallback.

To see this behavior in action, browse to the Retail Demo Store storefront using a different browser, an Incognito/Private window, or sign out of your existing account. What you should see on the home page is that instead of **"Inspired by your shopping behavior"**, the section is **"Popular products"**. After you click on a couple provide detail pages, return to the home page and see that the section title and recommendations have changed. This indicates that recommendations are now being personalized and will continue to become more relevant as you engage with products.

Similarly, the category pages will rerank products at first based on popularity and then become more and more personalized.

There are some challenges with this approach, though. First is the question of what to do with the provisional user ID when the user creates an account. To maintain continuity of the user's interaction history, the Retail Demo Store passes the provisional user ID to the Users microservice when creating a new user account. The Users service then uses this ID as the user's ID going forward. Another challenge is how to handle a user that anonymously browses the site using multiple devices such as on the mobile device and then on a desktop/laptop. In this case, separate provisional user IDs are generated for sessions on each device. However, once the user creates an account on one device and then signs in with that account on the other device, both devices will starting using the same user ID going forward. A side effect here is that the interaction history from one of the devices will be orphaned. This is an acceptable tradeoff given the benefit of cold starting users earlier and is functionally the same UX without this scheme. Additional logic could be added to merge the interaction history from both prior anonymous sessions when the user creates an account. Also, customer data platforms can be used to help manage this for you.

## Apply business rules to recommendations using filters

Often times recommendations need to be post-processed to apply business rules. For example, exclude products from being recommended that are no longer in stock or that the current user has recently purchased. Or creating a "Buy Again" user experience that only recommends products that the current user has recently purchased. Amazon Personalize provides this capability with a feature called [filters](https://docs.aws.amazon.com/personalize/latest/dg/filter.html). Below we will create a few filters for the Retail Demo Store that are used with different use-cases.

### Create/lookup filter utility function

The following python function will be used to create/lookup each of our filters. It uses the [CreateFilter](https://docs.aws.amazon.com/personalize/latest/dg/API_CreateFilter.html) API to attempt to create the filter and, if the filter already exists, it uses the [ListFilters](https://docs.aws.amazon.com/personalize/latest/dg/API_ListFilters.html) API to lookup the filter's ARN.

In [ ]:
def create_filter(filter_name: str, filter_expression: str) -> str:
    """ Utility function that conditionally creates/looks up a Personalize filter """
    filter_arn = None
    
    max_time = time.time() + 60*60 # 1 hours
    while time.time() < max_time and filter_arn is None:
        try:
            response = personalize.create_filter(
                name = filter_name,
                datasetGroupArn = dataset_group_arn,
                filterExpression = filter_expression
            )

            filter_arn = response['filterArn']
        except personalize.exceptions.ResourceAlreadyExistsException:
            print(f'You aready created a filter named "{filter_name}" for this dataset group, seemingly')
            paginator = personalize.get_paginator('list_filters')
            for paginate_result in paginator.paginate(datasetGroupArn = dataset_group_arn):
                for filter in paginate_result['Filters']:
                    if filter['name'] == filter_name:
                        filter_arn = filter['filterArn']
                        break
            if not filter_arn:
                raise Exception(f'Filter {filter_name} not found for dataset group; does it already exist in another dataset group')
        except ClientError as e:
            if e.response['Error']['Code'] == 'LimitExceededException':
                print('Too many filters being created; pausing and retrying...')
                time.sleep(15)    
                continue
            else:
                raise e
    
    print(f'Filter "{filter_name} ARN = {filter_arn}')
    return filter_arn

### Exclude purchased products filter

Depending on the products being sold, it can be a poor user experience to recommend products that a user has already purchased. For a case like this we will create a filter that excludes recently purchased products. We'll do this by creating a filter expression that excludes items that have an interaction with an event type of `Purchase` for the user.

> As noted earlier, the Retail Demo Store web application streams clickstream events to Personalize when the user performs various actions such as viewing and purchasing products. The filter created below allows us to use those events as exclusion criteria. See the [AnalyticsHandler.js](https://github.com/aws-samples/retail-demo-store/blob/master/src/web-ui/src/analytics/AnalyticsHandler.js) file for the code that sends clickstream events.

In [ ]:
exclude_purchased_filter_arn = create_filter(
    'retaildemostore-filter-exclude-purchased-products',
    'EXCLUDE itemId WHERE INTERACTIONS.event_type in ("Purchase")'
)

### Convenience store products filters

The Alexa curbside pickup use case provided by the Retail Demo Store is focused on convenience store style products (soda, pizza, chips, etc). In order to recommend convenience store products only, we will use filters that exclude products based on categories. Notice that the second convenience store filter uses a compound or multi-expression filter.

In [ ]:
exclude_non_cstore_filter_arn = create_filter(
    'retaildemostore-filter-cstore-products',
    'EXCLUDE ItemID WHERE ITEMS.CATEGORY_L1 NOT IN ("cold dispensed", "hot dispensed", "salty snacks", "food service")'
)

exclude_non_cstore_purchased_filter_arn = create_filter(
    'retaildemostore-filter-exclude-purchased-cstore-products',
    'EXCLUDE ItemID WHERE INTERACTIONS.event_type IN ("Purchase") | EXCLUDE ItemID WHERE ITEMS.CATEGORY_L1 IN ("cold dispensed", "hot dispensed", "salty snacks", "food service")'
)

### Include by category filter

The next filter that we will create is one that excludes purchased products and only includes products within one or more categories. This filter illustrates a dynamic filter where the values in the filter expression are passed in at inference-time.

In [ ]:
include_category_filter_arn = create_filter(
    'retaildemostore-filter-same-categories',
    'INCLUDE ItemID WHERE ITEMS.CATEGORY_L1 IN CurrentItem.CATEGORY_L1'
)

In [ ]:
exclude_purchased_include_category_filter_arn = create_filter(
    'retaildemostore-filter-exclude-purchased-include-categories',
    'EXCLUDE ItemID WHERE INTERACTIONS.event_type IN ("Purchase") | INCLUDE ItemID WHERE ITEMS.CATEGORY_L1 IN CurrentItem.CATEGORY_L1'
)

### Promotional filters

The last filters that we will create will be used to surface promotional products in recommendations for popular products and personalized products.

In [ ]:
promoted_items_filter_arn = create_filter(
    'retaildemostore-filter-promoted-items',
    'EXCLUDE ItemID WHERE INTERACTIONS.event_type IN ("Purchase") | INCLUDE ItemID WHERE ITEMS.PROMOTED IN ("Y")'
)

promoted_items_no_cstore_filter_arn = create_filter(
    'retaildemostore-filter-promoted-items-no-cstore',
    'EXCLUDE ItemID WHERE INTERACTIONS.event_type IN ("Purchase") | INCLUDE ItemID WHERE ITEMS.PROMOTED IN ("Y") AND ITEMS.CATEGORY_L1 NOT IN ("cold dispensed", "hot dispensed", "salty snacks", "food service")'
)

### Wait for filters to be created

The following cell will wait for our filters to be fully created and active. This should only take a minute or so.

In [ ]:
%%time

filter_arns = [ 
    exclude_purchased_filter_arn, 
    exclude_non_cstore_filter_arn, 
    exclude_non_cstore_purchased_filter_arn,
    include_category_filter_arn,
    exclude_purchased_include_category_filter_arn,
    promoted_items_filter_arn,
    promoted_items_no_cstore_filter_arn
]

max_time = time.time() + 60*60 # 1 hour
while time.time() < max_time:
    for filter_arn in reversed(filter_arns):
        response = personalize.describe_filter(
            filterArn = filter_arn
        )
        status = response["filter"]["status"]

        if status == "ACTIVE":
            print(f'Filter {filter_arn} successfully created')
            filter_arns.remove(filter_arn)
        elif status == "CREATE FAILED":
            print(f'Filter {filter_arn} failed')
            if response['filter'].get('failureReason'):
                print('   Reason: ' + response['filter']['failureReason'])
            filter_arns.remove(filter_arn)

    if len(filter_arns) > 0:
        print('At least one filter is still in progress')
        time.sleep(15)
    else:
        print("All filters have completed")
        break

### Test Purchased Products Filter

To test our purchased products filter, we will request recommendations for a random user. Then we will send an `Purchase` event for one of the recommended products to Personalize using the event tracker created above. Finally, we will request recommendations again for the same user but this time specify our filter.

In [ ]:
# Pick a user ID in the range of test users and fetch 5 recommendations.
user_id = '456'
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = rfy_recommender_arn,
    userId = user_id,
    numResults = 5
)

item_list = get_recommendations_response['itemList']
print(json.dumps(item_list, indent=2))

Next let's randomly select an item from the returned list of recommendations to be our product to purchase.

In [ ]:
product_id_to_purchase = random.choice(item_list)['itemId']
print(f'Product to simulate purchasing: {product_id_to_purchase}')

Next let's send an `Purchase` event to Personalize to simulate that the product was just purchased.
This will match the criteria for our filter.
In the Retail Demo Store web application, this event is sent for each product in the order after the order is completed.

In [ ]:
response = personalize_events.put_events(
    trackingId = event_tracking_id,
    userId = user_id,
    sessionId = str(uuid.uuid4()),
    eventList = [
        {
            'eventId': str(uuid.uuid4()),
            'eventType': 'Purchase',
            'itemId': str(product_id_to_purchase),
            'sentAt': int(time.time()),
            'properties': '{"discount": "No"}'
        }
    ]
)

# Wait for Purchase event to become consistent.
time.sleep(10)

print(json.dumps(response, indent=2))

Finally, let's retrieve recommendations for the user again but this time specifying the filter to exclude recently
purchased items. We do this by passing the filter's ARN via the `filterArn` parameter.
In the Retail Demo Store, this is done in the
[Recommendations](https://github.com/aws-samples/retail-demo-store/tree/master/src/recommendations) service.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = rfy_recommender_arn,
    userId = user_id,
    numResults = 5,
    filterArn = filter_arn
)

item_list = get_recommendations_response['itemList']
print(json.dumps(item_list, indent=2))

The following code will raise an assertion error if the product we just purchased is still recommended.

In [ ]:
found_item = next((item for item in item_list if item['itemId'] == product_id_to_purchase), None)
if found_item:
    assert found_item == False, 'Purchased item found unexpectedly in recommendations'
else:
    print('Purchased item filtered from recommendations for user!')

### Test Promotional filter

Next, let's test one of the promotional filters to see how they can be used to ensure that product meeting a specific filter are represented as a percentage of recommended products.

For this demo application, you may remember that the Items dataset (from Lab 1) has a column named `PROMOTED` that has a value `Y` or `N` for each product. One of the promotional filters that we created above will include products where `Items.PROMOTED IN ("Y")`. Let's see how we can use this promotional filter with the Recommended-For-You recommender.

Take note of the `promotions` parameter and the values passed in the `GetRecommendations` API call below. The name of the promotion (`promotions[].name`) is user-defined and will be echoed back in the response for promoted items. The `promotions[].filterArn` is the ARN for the promotional filter. And `promotions[].percentPromotedItems` indicates the percentage of recommended items should be the result of the promotional filter.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = rfy_recommender_arn,
    userId = user_id,
    numResults = 10,
    promotions = [
        {
            'name': 'my-promo',
            'filterArn': promoted_items_filter_arn,
            'percentPromotedItems': 30
        }
    ]
)

item_list = get_recommendations_response['itemList']
print(json.dumps(item_list, indent=2))

In [ ]:
promotion = {
    'name': 'my-promo',
    'filterArn': promoted_items_filter_arn,
    'percentPromotedItems': 30
}
df = get_recommendations_as_df(rfy_recommender_arn, user_id = user["id"], num_results = 15, promotion = promotion)
header = f'<h3>Recommended-For-You recommendations with Promotions for {user["first_name"]} {user["last_name"]} (#{user["id"]})</h3>'
header += f'<h4>Persona: {", ".join(user["persona"].split("_"))}</h4>'
HTML(header + df.to_html(escape=False))

### Update filter SSM parameters

With our filters created and one of them tested, the last step is to update the SSM parameters that is used throughout the Retail Demo Store project to detect and use the filter ARNs.

The [Recommendations](https://github.com/aws-samples/retail-demo-store/tree/master/src/recommendations) service already has logic to look for these filter ARNs in SSM and use them when fetching recommendations. All we have to do is set the filter ARNs in SSM.

In [ ]:
ssm.put_parameter(
    Name='/retaildemostore/personalize/filters/filter-purchased-arn',
    Description='Retail Demo Store Personalize Filter Purchased Products Arn Parameter',
    Value=exclude_purchased_filter_arn,
    Type='String',
    Overwrite=True
)

In [ ]:
ssm.put_parameter(
    Name='/retaildemostore/personalize/filters/filter-cstore-arn',
    Description='Retail Demo Store Filter C-Store Products Arn Parameter',
    Value=exclude_non_cstore_filter_arn,
    Type='String',
    Overwrite=True
)

In [ ]:
ssm.put_parameter(
    Name='/retaildemostore/personalize/filters/filter-purchased-and-cstore-arn',
    Description='Retail Demo Store Filter Purchased and C-Store Products Arn Parameter',
    Value=exclude_non_cstore_purchased_filter_arn,
    Type='String',
    Overwrite=True
)

In [ ]:
ssm.put_parameter(
    Name='/retaildemostore/personalize/filters/filter-same-categories-arn',
    Description='Retail Demo Store Filter to Include Same Categories Arn Parameter',
    Value=include_category_filter_arn,
    Type='String',
    Overwrite=True
)

In [ ]:
ssm.put_parameter(
    Name='/retaildemostore/personalize/filters/filter-exclude-purchased-include-categories-arn',
    Description='Retail Demo Store Filter to Exclude Purchased and Include Same Categories Arn Parameter',
    Value=exclude_purchased_include_category_filter_arn,
    Type='String',
    Overwrite=True
)

In [ ]:
ssm.put_parameter(
    Name='/retaildemostore/personalize/filters/promoted-items-filter-arn',
    Description='Retail Demo Store Promotional Filter to Include Promoted Items Arn Parameter',
    Value=promoted_items_filter_arn,
    Type='String',
    Overwrite=True
)

In [ ]:
ssm.put_parameter(
    Name='/retaildemostore/personalize/filters/promoted-items-no-cstore-filter-arn',
    Description='Retail Demo Store Promotional Filter to Include Promoted Non-CStore Items Arn Parameter',
    Value=promoted_items_no_cstore_filter_arn,
    Type='String',
    Overwrite=True
)

Now if you test completing an order for one or more items in the Retail Demo Store web application for a user,
those products should no longer be included in recommendations for that user.

Test it out by purchasing a recommended product from the "Inspired by your shopping trends"
section of the home page and then verifying that the product is no longer recommended.

Also be sure to test the "Compare similar items" recommendations on the product detail page for a few products. How have the recommendations been improved with the addition of the filter? Are they consistently more thematically similar?

Finally, check the recommendations on the homepage for "Promoted" items in both the "Popular products" (cold user) and "Inspired by your shopping trends" (warm/existing user) grid controls.

## Lab 4 Summary - What have we accomplished?

In this lab we evaluated the Personalize recommenders and campaigns created in the previous lab. We did this by directly calling the recommenders/campaigns using code and then using the Retail Demo Store web UI. Then we created a Personalize event tracker to collect real-time events/interactions and evaluated how it is used to update Personalize with user behavior and how recommendations change based on this behavior. Finally, we created and evaluated several filters to enforece specific business rules for the personalization use cases in the application.

This is the last core/required lab but there are several other labs that explore how to use additional Personalize features such as user segmentation and generative AI as well as how to use Personalize with various partner ISV solutions.

### Store variables needed in next labs

We will pass some variables initialized in this lab to subsequent labs by storing them in the notebook environment.

In [ ]:
# Store event tracker details
%store event_tracker_arn
%store event_tracking_id

# Store filter ARNs
%store exclude_purchased_filter_arn 
%store exclude_non_cstore_filter_arn
%store exclude_non_cstore_purchased_filter_arn
%store include_category_filter_arn
%store exclude_purchased_include_category_filter_arn
%store promoted_items_filter_arn
%store promoted_items_no_cstore_filter_arn

## Core workshop Complete

Congratulations! You have completed the core Retail Demo Store Personalization Workshop. There are several other labs that explore how to use additional Personalize features such as user segmentation and generative AI as well as how to use Personalize with various partner ISV solutions.

### Cleanup (if you're done)

If you launched the Retail Demo Store in your personal AWS account **AND** you're done with all workshops, you can follow the [Personalize workshop cleanup](./Lab-10-(Optional)-Cleanup-Personalize-resources.ipynb) notebook to delete all of the Amazon Personalize resources created by this workshop. If you plan on completing an optional personalization lab or other workshops that depend on Personalize, do not perform the Personalize cleanup at this time. **IMPORTANT: since the Personalize resources were created by this notebook and not CloudFormation, deleting the CloudFormation stack for the Retail Demo Store will not remove the Personalize resources. You MUST run the [Personalize workshop cleanup](./Lab-10-(Optional)-Cleanup-Personalize-resources.ipynb) notebook or manually clean up these resources.**

If you are participating in an AWS managed event such as a workshop and using an AWS provided temporary account, you can skip the cleanup workshop unless otherwise instructed.